In [4]:
import json, joblib, numpy as np, pandas as pd, tensorflow as tf
from sklearn.metrics import classification_report

# ==== Dosyaları yükle ====
MODEL_PATH   = "ae_fraud_best.keras"
PREPROC_PKL  = "preprocess_ae.joblib"
THR_JSON     = "ae_thresholds.json"  # {'threshold_f1': ..., 'threshold_p90': ...}

ae         = tf.keras.models.load_model(MODEL_PATH)
preprocess = joblib.load(PREPROC_PKL)
thr_dict   = json.load(open(THR_JSON))
thr_f1     = float(thr_dict["threshold_f1"])
thr_p90    = float(thr_dict["threshold_p90"])

# ==== Veri ====
CSV_PATH     = "Credit_Card_Applications.csv"   # test etmek istediğin dosya
ID_COL       = "CustomerID"                     # varsa kimlik kolonu
TARGET_COL   = "Class"                          # varsa gerçek etiket (0=normal,1=anomali)

df = pd.read_csv(CSV_PATH)

# Özellik matrisini hazırla (ID/target hariç)
drop_cols = [c for c in [ID_COL, TARGET_COL] if c in df.columns]
X_df = df.drop(columns=drop_cols)

# Eğitimdeki sırayla aynı dönüşümler:
Xp   = preprocess.transform(X_df)
Xhat = ae.predict(Xp, verbose=0)

# Rekonstrüksiyon hatası (MAE) = anomali skoru
scores = np.mean(np.abs(Xp - Xhat), axis=1)

# İki farklı eşikle tahmin
pred_f1  = (scores >= thr_f1 ).astype(int)
pred_p90 = (scores >= thr_p90).astype(int)

# Çıktı tablosu
out = pd.DataFrame({
    "ID": df[ID_COL].values if ID_COL in df.columns else np.arange(len(df)),
    "score": scores,
    "pred_f1": pred_f1,
    "pred_p90": pred_p90,
})
# Top-10 en anomali kayıt
top10 = out.sort_values("score", ascending=False).head(10)
top10_full = top10.merge(df, left_on="ID", right_on=ID_COL, how="left")
top10_full.to_csv("top_anomalies.csv", index=False)

print("Top-10 anomaliler (skora göre):")
print(top10[["ID","score","pred_f1","pred_p90"]])

# Etiket varsa kısaca metrik yazdır
if TARGET_COL in df.columns:
    y_true = df[TARGET_COL].values.astype(int)
    print("\n== F1-opt eşiği ile ==")
    print(classification_report(y_true, pred_f1, digits=3))
    print("== P>=0.90 eşiği ile ==")
    print(classification_report(y_true, pred_p90, digits=3))

print("\nKaydedildi: top_anomalies.csv")
print(f"Eşikler -> F1-opt: {thr_f1:.6f} | P>=0.90: {thr_p90:.6f}")


Top-10 anomaliler (skora göre):
           ID      score  pred_f1  pred_p90
500  15790113  11.726988        1         1
149  15598802   6.051972        1         1
267  15762716   5.767589        1         1
202  15664615   3.767000        1         1
196  15708714   3.129059        1         1
559  15642098   2.264231        1         1
27   15588123   2.224084        1         1
240  15772329   1.794907        1         1
551  15801817   1.724704        1         1
154  15735837   1.641210        1         1

== F1-opt eşiği ile ==
              precision    recall  f1-score   support

           0      0.806     0.739     0.771       383
           1      0.705     0.779     0.740       307

    accuracy                          0.757       690
   macro avg      0.756     0.759     0.756       690
weighted avg      0.761     0.757     0.757       690

== P>=0.90 eşiği ile ==
              precision    recall  f1-score   support

           0      0.604     0.990     0.750       383
